# Préparation de l'espace de travail

## Importations

In [ ]:
import openpyxl 
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## Lecture des bases de données
### Base de la démographie des médecins : medecin

In [ ]:
# Lecture des différentes feuilles du Excel medecins
df_medecinsEffectif = pd.read_excel("Bases de données/medecins.xlsx", sheet_name=1)
df_medecinsAge = pd.read_excel("Bases de données/medecins.xlsx", sheet_name=2)
df_medecinsDensite = pd.read_excel("Bases de données/medecins.xlsx", sheet_name=3)
df_medecinsEffectifsParSpe = pd.read_excel("Bases de données/medecins.xlsx", sheet_name=4)
df_medecinsEffectifsParLieu = pd.read_excel("Bases de données/medecins.xlsx", sheet_name=5)
df_medecinsNombreActivites = pd.read_excel("Bases de données/medecins.xlsx", sheet_name=6)

### Base de la prévalence de différentes maladies par groupe de population : santePublique

In [ ]:
# Lecture du csv santePublique
df_santePublique = pd.read_csv("Bases de données/santePublique.csv", sep=";")

### Base de l'indicateur APL (Accessibilité Potentielle Localisée) pour les médecins généralistes 

In [ ]:
# Lecture du Excel APL_2022_2023 : tableau de la pondération de la population
# (consommation moyenne en soins de la tranche d'âge rapportée à la consommation moyenne de la population)
df_pond_population_2022_2023 = pd.read_excel("Bases de données/APL_2022_2023.xlsx", sheet_name=0)[21:40]
df_pond_population_2022_2023.reset_index(drop=True, inplace=True)

# Renommage des colonnes
df_pond_population_2022_2023.columns = ["Tranche d'âge", "Poids de la tranche d'âge en 2022", "Poids de la tranche d'âge en 2023"]

# Lecture du Excel APL_2015_2022 : tableau de pondération de la population (on exclut l'année 2022 en supposant que les données de APL_2022_2023 sont plus pertinentes)
df_pond_population_2015_2021 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=0)[21:40]
df_pond_population_2015_2021.reset_index(drop=True, inplace=True)

# Renommage des colonnes
df_pond_population_2015_2021.columns = ["Tranche d'âge", "Poids de la tranche d'âge en 2015", "Poids de la tranche d'âge en 2016", "Poids de la tranche d'âge en 2017", 
    "Poids de la tranche d'âge en 2018", "Poids de la tranche d'âge en 2019", "Poids de la tranche d'âge en 2021", "Poids de la tranche d'âge en 2022 bis"]
df_pond_population_2015_2021.drop("Poids de la tranche d'âge en 2022 bis", axis=1, inplace=True)

# Merging des deux tables sur la clé de la tranche d'âge
df_pond_population = df_pond_population_2015_2021.merge(right=df_pond_population_2022_2023, how="left", on=["Tranche d'âge"])

df_pond_population

In [31]:
# Lecture des deux feuilles de la table APL_2022_2023
df_APL_2022 = pd.read_excel("Bases de données/APL_2022_2023.xlsx", sheet_name=1)[8:]
df_APL_2023 = pd.read_excel("Bases de données/APL_2022_2023.xlsx", sheet_name=2)[8:]

# Nettoyage et renommage des colonnes
bases = [df_APL_2022, df_APL_2023]
annee = 2022

for base in bases : 
    base.drop(8,inplace=True)
    base.reset_index(drop=True, inplace=True)
    base.columns = ["Code commune INSEE", "Commune", f"APL_{annee}", f"APL_{annee}_moins_65", f"APL_{annee}_moins_62", f"APL_{annee}_moins_60", f"population_standard_{annee-2}", f"population_totale_{annee-2}"]
    annee += 1

# Lecture des feuilles de la table APL_2015_2022 (on exclut l'année 2022 en supposant que les données de APL_2022_2023 sont plus pertinentes)
df_APL_2015 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=1)[8:]
df_APL_2016 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=2)[8:]
df_APL_2017 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=3)[8:]
df_APL_2018 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=4)[8:]
df_APL_2019 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=5)[8:]
df_APL_2021 = pd.read_excel("Bases de données/APL_2015_2022.xlsx", sheet_name=6)[8:]

# Nettoyage et renommage des colonnes
bases = [df_APL_2015,df_APL_2016,df_APL_2017,df_APL_2018,df_APL_2019,df_APL_2021]
annee = 2015

for base in bases : 
    base.drop(8,inplace=True)
    base.reset_index(drop=True, inplace=True)
    base.columns = ["Code commune INSEE", "Commune", f"APL_{annee}", f"APL_{annee}_moins_65", f"population_standard_{annee-2}", f"population_totale_{annee-2}"]
    annee += 1
    if annee == 2020 : 
        annee += 1

bases.append(df_APL_2022)
df_APL = df_APL_2023
for base in bases : 
    df_APL = df_APL.merge(base, how='left', on=["Code commune INSEE", "Commune"])

df_APL

,Code commune INSEE,Commune,APL_2023,APL_2023_moins_65,APL_2023_moins_62,APL_2023_moins_60,population_standard_2021,population_totale_2021,APL_2015,APL_2015_moins_65,...,APL_2021,APL_2021_moins_65,population_standard_2019,population_totale_2019,APL_2022,APL_2022_moins_65,APL_2022_moins_62,APL_2022_moins_60,population_standard_2020,population_totale_2020
0,01001,L'Abergement-Clémenciat,1.942,1.881,1.623,1.455,838.154,832,NaN,NaN,...,2.15,2.076,775.023,779,1.89,1.828,1.799,1.491,810.855,806
1,01002,L'Abergement-de-Varey,2.376,1.767,1.503,1.333,255.723,267,NaN,NaN,...,2.505,2.19,252.742,256,2.649,2.059,1.83,1.493,250.962,262
2,01004,Ambérieu-en-Bugey,3.083,2.431,2.136,1.855,14575.887,14854,4.42,4.324,...,3.367,3.102,13796.994,14134,3.646,2.88,2.697,2.253,14002.685,14288
3,01005,Ambérieux-en-Dombes,3.706,3.648,3.015,2.998,1852.496,1897,4.24,4.24,...,4.303,4.303,1717.427,1751,3.978,3.959,3.312,3.041,1733.232,1782
4,01006,Ambléon,0.889,0.775,0.648,0.57,121.314,113,1.103,0.926,...,0.917,0.793,117.329,112,0.926,0.796,0.746,0.588,121.438,113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34949,97420,Sainte-Suzanne,3.764,3.235,2.64,2.36,22314.19,24293,4.269,3.799,...,3.732,3.156,22206.544,24065,3.701,3.096,2.588,2.151,21899.922,24086
34950,97421,Salazie,3.354,2.465,1.576,1.576,6748.4,7243,2.079,1.774,...,4.167,3.276,6736.065,7136,4.081,3.194,3.194,2.308,6768.561,7310
34951,97422,Le Tampon,5.475,4.497,4.09,3.643,76427.591,81943,5.215,4.616,...,5.29,4.509,75245.866,79824,5.422,4.575,4.131,3.638,74961.891,80778
34952,97423,Les Trois-Bassins,5.044,5.044,5.044,5.044,6571.416,6899,3.144,2.993,...,5.629,4.954,6716.635,7015,6.507,5.367,5.367,5.367,6596.222,6953
